# Bias

## Laboratorio in Python

### Esperimento 1: Bias di Selezione

Simuliamo un dataset dove la popolazione rappresentata è squilibrata tra due gruppi. Addestriamo un modello e analizziamo l'accuratezza per ciascun gruppo.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
import seaborn as sns

# Simulazione di un dataset con bias di selezione
np.random.seed(42)
n_samples = 500

# Gruppo A (es. uomini)
X_A = np.random.normal(loc=50, scale=10, size=(int(n_samples * 0.8), 1))  # più rappresentato
y_A = (X_A.flatten() > 50).astype(int)  # outcome dipende dal valore

# Gruppo B (es. donne), sottorappresentato
X_B = np.random.normal(loc=50, scale=10, size=(int(n_samples * 0.2), 1))
y_B = (X_B.flatten() > 50).astype(int)

# Combinazione dei dati
X = np.vstack((X_A, X_B))
y = np.concatenate((y_A, y_B))
groups = np.array(['A'] * len(X_A) + ['B'] * len(X_B))

# Divisione del dataset
X_train, X_test, y_train, y_test, groups_train, groups_test = train_test_split(X, y, groups, test_size=0.3, random_state=42)

# Addestramento modello
model = LogisticRegression()
model.fit(X_train, y_train)

# Predizioni
y_pred = model.predict(X_test)

# Accuratezza complessiva
acc = accuracy_score(y_test, y_pred)

# Accuratezza per gruppo
acc_A = accuracy_score(y_test[groups_test == 'A'], y_pred[groups_test == 'A'])
acc_B = accuracy_score(y_test[groups_test == 'B'], y_pred[groups_test == 'B'])

# Matrice di confusione
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title("Matrice di Confusione - Bias di Selezione")
plt.show()

# Visualizzazione delle accuratezze
sns.barplot(x=['Totale', 'Gruppo A', 'Gruppo B'], y=[acc, acc_A, acc_B])
plt.title('Accuratezza per Gruppo')
plt.ylabel('Accuratezza')
plt.ylim(0, 1)
plt.grid(True)
plt.show()

acc, acc_A, acc_B

I risultati dell'esperimento sul **bias di selezione** sono i seguenti:

* **Accuratezza complessiva**: 99.3%
* **Accuratezza sul Gruppo A** (maggiormente rappresentato): 100%
* **Accuratezza sul Gruppo B** (sottorappresentato): 96.8%

Questo esperimento evidenzia il pericolo del **bias di selezione**, un problema comune nei sistemi di classificazione. Anche se il modello ha un'elevata accuratezza complessiva, la sua performance sul gruppo sottorappresentato è significativamente inferiore. Questo dimostra come un modello possa essere ingiusto verso alcune categorie, anche se sembra essere accurato in generale.

### Esperimento 2: Bias Sistemico

L'obiettivo di questo esperimento è dimostrare l’effetto di un bias sistemico di genere in un modello di selezione del personale.
A questo scopo simuliamo un processo di selezione del personale in cui, a parità di esperienza e età, **le donne vengono penalizzate** sistematicamente rispetto agli uomini. Usiamo un dataset artificiale in cui il genere influisce negativamente sulla probabilità di assunzione, creando così un **bias sistemico**.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Generazione dati simulati
np.random.seed(42)
n = 1000
esperienza = np.random.normal(5, 2, n)
eta = np.random.normal(35, 5, n)
genere = np.random.choice([0, 1], size=n)  # 0 = femmina, 1 = maschio

# Bias sistemico: penalizzazione delle donne (genere=0)
logits = 0.5 * esperienza + 0.1 * eta + 1.0 * genere - 7.5
prob_assunzione = 1 / (1 + np.exp(-logits))
assunto = np.random.binomial(1, prob_assunzione)

# Dataset
df = pd.DataFrame({
    'esperienza': esperienza,
    'eta': eta,
    'genere': genere,
    'assunto': assunto
})

# Split e normalizzazione
X = df[['esperienza', 'eta', 'genere']]
y = df['assunto']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Modello
model = LogisticRegression()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

# Classificazione report
report = classification_report(y_test, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()

# Grafico: distribuzione delle probabilità predette per maschi e femmine
prob_pred = model.predict_proba(X_test_scaled)[:, 1]
X_test_with_probs = pd.DataFrame(X_test_scaled, columns=['esperienza', 'eta', 'genere'])
X_test_with_probs['prob_assunto'] = prob_pred
X_test_with_probs['genere'] = X_test['genere'].values

# Plot
plt.figure(figsize=(10, 6))
for g, label in zip([0, 1], ['Femmine', 'Maschi']):
    subset = X_test_with_probs[X_test_with_probs['genere'] == g]
    plt.hist(subset['prob_assunto'], bins=25, alpha=0.6, label=label, density=True)
plt.xlabel('Probabilità Predetta di Assunzione')
plt.ylabel('Densità')
plt.title('Distribuzione della Probabilità di Assunzione per Genere')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

1. **Importazione delle librerie**

```python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import train_test_split
```

Queste librerie permettono di:

* creare e gestire dati (`numpy`, `pandas`)
* visualizzare grafici (`matplotlib`)
* creare e valutare un modello di regressione logistica (`sklearn`)

---

2. **Generazione del dataset simulato**

```python
np.random.seed(42)
n = 1000
genere = np.random.binomial(1, 0.5, n)  # 0=f, 1=m
esperienza = np.random.normal(5 + genere*1.5, 1, n)  # maschi con +1.5 anni
eta = np.random.normal(35, 5, n)
```

Qui generiamo 1000 candidati:

* `genere`: 0 = femmina, 1 = maschio, distribuiti equamente.
* `esperienza`: in media, i maschi hanno più esperienza (bias indotto).
* `età`: distribuita normalmente.

```python
bias = -0.8 * (1 - genere)  # penalizzazione per le donne
logits = 0.3*esperienza + 0.05*eta + bias
prob = 1 / (1 + np.exp(-logits))
assunto = np.random.binomial(1, prob)
```

* Calcoliamo la **probabilità di assunzione** come combinazione lineare delle variabili, con un **bias penalizzante per le donne**.
* La variabile `assunto` simula se il candidato viene assunto.

3. **Addestramento del modello**

```python
X = np.column_stack((esperienza, eta, genere))
y = assunto
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
```

* Addestriamo un modello di **regressione logistica** per predire l’assunzione.
* Usiamo il **genere** come input, quindi il modello può apprendere e replicare il bias.

4. **Visualizzazione del bias**

```python
X_df = pd.DataFrame(X_test, columns=["esperienza", "eta", "genere"])
X_df["prob_pred"] = model.predict_proba(X_test)[:,1]
X_df["genere"] = X_df["genere"].astype(int)

plt.figure(figsize=(8, 6))
for g in [0, 1]:
    subset = X_df[X_df["genere"] == g]
    label = "Femmine" if g == 0 else "Maschi"
    plt.hist(subset["prob_pred"], bins=20, alpha=0.5, label=label, density=True)

plt.title("Distribuzione delle probabilità predette di assunzione per genere")
plt.xlabel("Probabilità predetta di assunzione")
plt.ylabel("Densità")
plt.legend()
plt.grid(True)
plt.show()
```

* **Istogramma** delle probabilità predette per ciascun genere.
* Mostra chiaramente che i **maschi** hanno **più probabilità di essere assunti**, anche a parità di altre condizioni.

Questo esperimento dimostra visivamente e quantitativamente:

* l’effetto del **bias sistemico**,
* come esso possa essere **appreso e amplificato** da un modello,
* perché sia cruciale **rimuovere variabili sensibili** o applicare **strategie di mitigazione**.

### Esperimento 2: Bias di razza nella valutazione del rischio di recidiva

Obiettivo di questo esperimento è dimostrare come un modello predittivo possa riflettere o amplificare bias razziali analizzando la distribuzione delle probabilità predette per la recidiva in due gruppi demografici distinti.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# 1. Simulazione dataset giustizia penale con possibile bias razziale
np.random.seed(42)
n = 1000
# Variabile sensibile: razza (0=gruppo A, 1=gruppo B)
razza = np.random.binomial(1, 0.5, n)

# Caratteristiche: numero precedenti penali, età
precedenti = np.random.poisson(2 + razza * 0.5, n)  # il gruppo B ha in media più precedenti
eta = np.random.normal(35, 5, n)

# Probabilità "vera" di recidiva senza bias
logits_veri = 0.8 * precedenti + 0.02 * eta
prob_vera = 1 / (1 + np.exp(-logits_veri))
recidiva_reale = np.random.binomial(1, prob_vera)

# Sistema giudiziario introduce un bias implicito (più severo con gruppo B)
bias = 0.7 * razza
logits_predetti = logits_veri + bias
prob_predetta = 1 / (1 + np.exp(-logits_predetti))
recidiva_predetta = np.random.binomial(1, prob_predetta)

# 2. Addestramento modello su dati "biasati"
X = np.column_stack((precedenti, eta, razza))
y = recidiva_predetta
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
probs = model.predict_proba(X_test)[:, 1]

# 3. Analisi dell'equità: confronto distribuzioni delle probabilità predette per gruppo
X_test_df = pd.DataFrame(X_test, columns=["precedenti", "eta", "razza"])
X_test_df["prob_pred"] = probs
X_test_df["razza"] = X_test_df["razza"].astype(int)

# 4. Grafico della distribuzione predetta per razza
plt.figure(figsize=(8, 6))
for r in [0, 1]:
    subset = X_test_df[X_test_df["razza"] == r]
    label = "Gruppo A" if r == 0 else "Gruppo B"
    plt.hist(subset["prob_pred"], bins=20, alpha=0.6, label=label, density=True)

plt.title("Distribuzione delle probabilità predette di recidiva per gruppo razziale")
plt.xlabel("Probabilità predetta di recidiva")
plt.ylabel("Densità")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 5. Rapporto di classificazione
report = classification_report(y_test, y_pred, target_names=["No recidiva", "Recidiva"])
report

Il codice Python simula un dataset di giustizia penale con possibile bias razziale. Il modello predittivo apprende e amplifica questo bias, mostrando come la distribuzione delle probabilità predette sia diversa per i due gruppi.

1. **Simulazione dei dati**

Abbiamo simulato un dataset di **1.000 soggetti** nel contesto della giustizia penale, con le seguenti caratteristiche:

* **Razza** (`0=Gruppo A`, `1=Gruppo B`)
* **Numero di precedenti penali**
* **Età**

La **variabile sensibile** è la razza. Abbiamo introdotto un *bias implicito* che aumenta il rischio predetto di recidiva per il gruppo B, anche a parità di condizioni con il gruppo A.

2. **Addestramento del modello**

È stato addestrato un modello di **regressione logistica** sui dati biasati per simulare l’apprendimento in un ambiente non equo.

3. **Analisi dell’equità**

Abbiamo calcolato le **probabilità predette** di recidiva per ogni soggetto nel test set e confrontato graficamente le distribuzioni nei due gruppi razziali.

4. **Risultati**

Il grafico mostra chiaramente che il **gruppo B riceve sistematicamente predizioni di rischio più alte**, anche a parità di precedenti e di età. Questo è un segnale diretto di **bias sistemico** nel processo predittivo.

5. **Valutazione delle prestazioni**

Dal report di classificazione:

* Il modello ha una **accuratezza del 91%**, ma è totalmente sbilanciato: **non predice mai la classe "No recidiva"** (precision = 0.00, recall = 0.00).
* Questo è un esempio perfetto di come un modello possa sembrare “accurato” ma essere **profondamente ingiusto**.



L'esperimento dimostra che:

* Il bias può essere introdotto **nei dati**, non necessariamente nel modello.
* È fondamentale **analizzare le predizioni per sottogruppi** (e.g., gruppi razziali, genere, età).
* Anche modelli con alta accuratezza possono **comportarsi in modo discriminatorio**, e le metriche aggregate possono mascherare queste disuguaglianze.